Variables:

1. Monthly returns including dividend distributions

ret

2. Share Price 

prc

3. Common Shares Outstanding 

shrout

4. Share code  

shrcd

5. CRSP permanent company identifier (Permno and or Permco) 

permco 

6. Delisting return 

dlret

7. Date 

8. CRSP value‐weighted index returns

vwretd

In [104]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import regex as re
import statsmodels.formula.api as smf

crsp = pd.read_csv('/Users/kailiao/Downloads/crsp.csv', dtype = object)

# change strings or floats to integers
for col in ['PERMNO','PERMCO']:
    crsp[col] = crsp[col].astype(int)

# change returns and prices to floats. If error, coerce to NaN
for col in ['PRC', 'RET', 'vwretd', 'SHROUT'] :
    crsp[col] = pd.to_numeric(crsp[col], errors='coerce')

# deal with negative price
crsp['PRC'] = crsp['PRC'].abs()

crsp['year']  = crsp['date'].str.slice(start=0, stop=4).astype(int)
crsp['month'] = crsp['date'].str.slice(start=4, stop=6).astype(int)
crsp['mrkcap'] = crsp['PRC'] * crsp['SHROUT']

In [105]:
periods = [1973 + i for i in np.arange(9) * 5]
periods

[1973, 1978, 1983, 1988, 1993, 1998, 2003, 2008, 2013]

In [134]:
pd.options.mode.chained_assignment = None

period= 1973
crsp_1 = crsp[(crsp['year'] < period + 3) & (crsp['year'] >= period)]

def kill_nan_ret(df):
    if df['RET'].isnull().values.any():
        df['mrkcap'] = 0
    return df

crsp_1 = crsp_1.groupby('PERMNO').apply(kill_nan_ret)

# select the largest 400 firms
largest = list(crsp_1.groupby('PERMNO').first().nlargest(n=500, columns=['mrkcap']).reset_index().PERMNO)
crsp_2 = pd.DataFrame()
for firm in largest:
    crsp_temp = crsp_1[crsp_1['PERMNO'] == firm]
    model = smf.ols("RET ~ vwretd", data=crsp_temp)
    result = model.fit()
    crsp_temp['beta'] = result.params.vwretd
    crsp_2 = pd.concat([crsp_2, crsp_temp])

crsp_2 = crsp_2.merge(crsp_2.groupby('PERMNO').first().beta.rank().reset_index(), on='PERMNO')
crsp_2 = crsp_2.rename(columns={"beta_x" : "beta", "beta_y" : "beta_rank"})


In [139]:
crsp_2[crsp_2['beta'] == 124]

    


,PERMNO,date,SHRCD,TICKER,PERMCO,DLRET,PRC,RET,SHROUT,vwretd,year,month,mrkcap,beta,beta_rank


In [74]:
#import ML packages
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

crsp_2 = crsp_1[crsp_1['PERMNO'] == 93172]
crsp_2 

,PERMNO,date,SHRCD,TICKER,PERMCO,DLRET,PRC,RET,SHROUT,vwretd,year,month
4228046,93172,19730131,48,BULDS,699,NaN,-31.0625,-0.015842,2844,-0.027454,1973,1
4228047,93172,19730228,48,BULDS,699,NaN,-28.9375,-0.068410,2844,-0.044370,1973,2
4228048,93172,19730330,48,BSG,699,NaN,29.0000,0.002160,2495,-0.007857,1973,3
4228049,93172,19730430,48,BSG,699,NaN,29.0000,0.027586,2495,-0.051769,1973,4
4228050,93172,19730531,48,BSG,699,NaN,27.5000,-0.051724,2495,-0.024508,1973,5
4228051,93172,19730629,48,BSG,699,NaN,26.7500,-0.027273,2495,-0.008648,1973,6
4228052,93172,19730731,48,BSG,699,NaN,29.8750,0.150467,2495,0.056961,1973,7
4228053,93172,19730831,48,BSG,699,NaN,29.7500,-0.004184,2495,-0.029657,1973,8
4228054,93172,19730928,48,BSG,699,NaN,31.6250,0.063025,2495,0.053931,1973,9
4228055,93172,19731031,48,BSG,699,NaN,32.5000,0.059921,2495,-0.000415,1973,10


In [ ]:
y = crsp_2[['RET']].values
X = crsp_2[['vwretd']].values
LinearRegression().fit(X, y)

In [16]:
np.arange(9) * 5

array([ 0,  5, 10, 15, 20, 25, 30, 35, 40])

In [13]:
import numpy as np
import pandas as pd
import wrds       
conn = wrds.Connection() 

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
Loading library list...
Done


In [14]:
crsp = conn.raw_sql(
    """
    select *
    from crsp.msf a inner join crsp.msenames b
    on a.permno=b.permno and a.date between b.namedt and b.nameendt
    and b.exchcd in (1,2,3) and b.shrcd in (10,11)
    where a.date >= '2000-01-01'
    order by a.permno, a.date
    """,
date_cols=['date']
)

# change strings or floats to integers
for col in ['permno','permco']:
    crsp[col] = crsp[col].astype(int)

# define market equity as sum of market equities of all permnos associated with a permco
crsp['me'] = crsp.groupby(['date','permco']).me.transform(sum)

ValueError: Grouper for 'permco' not 1-dimensional

In [8]:
crsp = conn.raw_sql(
    """
    select a.permno, a.permco, a.date, a.ret, abs(a.prc)*a.shrout as me, a.vwretd, b.exchcd
    from crsp.msf a inner join crsp.msenames b
    on a.permno=b.permno and a.date between b.namedt and b.nameendt
    and b.exchcd in (1,2,3) and b.shrcd in (10,11)
    where a.date >= '2000-01-01'
    order by a.permno, a.date
    """,
date_cols=['date']
)

# change strings or floats to integers
for col in ['permno','permco']:
    crsp[col] = crsp[col].astype(int)

# define market equity as sum of market equities of all permnos associated with a permco
crsp['me'] = crsp.groupby(['date','permco']).me.transform(sum)

StatementError: (builtins.AttributeError) 'NoneType' object has no attribute 'cursor'
[SQL: 
    select a.permno, a.permco, a.date, a.ret, abs(a.prc)*a.shrout as me, a.vwretd, b.exchcd
    from crsp.msf a inner join crsp.msenames b
    on a.permno=b.permno and a.date between b.namedt and b.nameendt
    and b.exchcd in (1,2,3) and b.shrcd in (10,11)
    where a.date >= '2000-01-01'
    order by a.permno, a.date
    ]